# Softmax Classification

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# For reproducibility

torch.manual_seed(1)
# 항상 같은 결과를 유지하기 위해

### Discrete Probability Distribution (이산적 확률 분포)

연속확률분포 vs 이산확률분포 (PMF)

PDF(Probabillity Density Function) 함수경우에는 연속적인 확률분포를 가진다..
![](http://www.birc.co.kr/wp-content/uploads/2017/02/image.png)

여기서 x축의 어떤값을 딱 집을때 이곳은 확률이 될 수 없다.
P(X=x) = ?????

하지만 

![](https://t1.daumcdn.net/cfile/tistory/230C694255F56A8305)

이산확률분포의 어떤점 x는 확률을 갖는다. 

EX) P(X=6) = 1/6 , P(X=가위) = 0.33333 등


### Softmax
Convert unber to porbability with softamx

P(class = i) = exp(i)/sum(exp(i_t)

In [4]:
z = torch.FloatTensor([1, 2, 3])

Pytorch has a softmax function

In [5]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [6]:
hypothesis.sum()

tensor(1.)

### Cross Entropy

두개의 확률분포가 얼마나 비슷한지에 대해 알기 위함

우리가 가지고 있는 모델의 확률분포 함수는 점점 P에 근사하게 해야한다.
따라서 크로스 앤트로미 미니마이즈 하는것으 ㄹ최소화 해야한다

### Cross Entropy Loss(Low - Level)

For multi-classification, we use the cross entropyloss
L = 1/n(sum(-y*log(hatY)))

where y^ is the predicted probability(0 or 1)

In [8]:
z = torch.rand(3,5, requires_grad=True) # class 갯수 = 5 # samples 수 = 3
hypothesis = F.softmax(z, dim= 1)
print(hypothesis)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward>)


In [10]:
y = torch.randint(5,(3,)).long()
print(y)

tensor([3, 2, 3])


In [30]:
y_one_hot = torch.zeros_like(hypothesis)

In [31]:
y_one_hot.scatter_(1,y.unsqueeze(1),1)

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])

y_one_hot.scatter_(1,y.unsqueeze(1),1)  
__.sacatter___ : 아래 언더바는 inplace 연산을 하라는 뜻이다.

In [14]:
y.unsqueeze(1)

tensor([[3],
        [2],
        [3]])

In [15]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6800, grad_fn=<MeanBackward0>)


### Cross-entropy Loss with torch.nn functional

In [16]:
# Low Level

torch.log(F.softmax(z, dim=1))

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward>)

In [17]:
# High Level

F.log_softmax(z, dim=1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward>)

In [18]:
# Low Level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.6800, grad_fn=<MeanBackward0>)

In [23]:
# High Level
# NLL = Negative Log Likelihood
# 위의 sum, maen 과정을 생략해준다.

F.nll_loss(F.log_softmax(z, dim=1),y)

#log_softmax 는 -torch.log(F.softmax(z, dim=1))) 과정을 F.log_softmax(z, dim=1) 로 줄여준다.

tensor(1.6800, grad_fn=<NllLossBackward>)

Pytorch also has __F.cross_entropy__ that combines __F.log_softmax()__ and __F.null_loss()__

In [22]:
F.cross_entropy(z, y)

tensor(1.6800, grad_fn=<NllLossBackward>)

# Training with Low-Level Cross Entropy Loss

In [28]:
x_train = [[1,2,1,1],
           [2,1,3,2],
           [3,1,3,4],
           [4,1,5,5],
           [1,7,5,5],
           [1,2,5,6],
           [1,6,6,6],
           [1,7,7,7]
          ]

y_train = [2,2,2,1,1,1,0,0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [29]:
print(x_train.shape) #4차원의 class
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [35]:
# 모델 초기화
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# sample갯수 = m
# classes 갯수 = 3
# 입력 vec size = 4
# 따라서 우리는 4->3으로 가는 linear layer를 만드는 중이다.


# optimizer 설정
optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 100000
for epoch in range(nb_epochs + 1) :
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    
    
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    
    
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim = 1))).sum(dim=1).mean()
    
    # cost로 H(x1) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
     
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch,nb_epochs,cost.item()))
        

Epoch    0/100000 Cost: 1.098612
Epoch  100/100000 Cost: 0.901535
Epoch  200/100000 Cost: 0.839114
Epoch  300/100000 Cost: 0.807826
Epoch  400/100000 Cost: 0.788472
Epoch  500/100000 Cost: 0.774822
Epoch  600/100000 Cost: 0.764449
Epoch  700/100000 Cost: 0.756191
Epoch  800/100000 Cost: 0.749398
Epoch  900/100000 Cost: 0.743671
Epoch 1000/100000 Cost: 0.738749
Epoch 1100/100000 Cost: 0.734453
Epoch 1200/100000 Cost: 0.730657
Epoch 1300/100000 Cost: 0.727271
Epoch 1400/100000 Cost: 0.724223
Epoch 1500/100000 Cost: 0.721463
Epoch 1600/100000 Cost: 0.718948
Epoch 1700/100000 Cost: 0.716644
Epoch 1800/100000 Cost: 0.714526
Epoch 1900/100000 Cost: 0.712570
Epoch 2000/100000 Cost: 0.710759
Epoch 2100/100000 Cost: 0.709075
Epoch 2200/100000 Cost: 0.707507
Epoch 2300/100000 Cost: 0.706043
Epoch 2400/100000 Cost: 0.704672
Epoch 2500/100000 Cost: 0.703385
Epoch 2600/100000 Cost: 0.702177
Epoch 2700/100000 Cost: 0.701038
Epoch 2800/100000 Cost: 0.699964
Epoch 2900/100000 Cost: 0.698950
Epoch 3000

Epoch 24300/100000 Cost: 0.669248
Epoch 24400/100000 Cost: 0.669230
Epoch 24500/100000 Cost: 0.669211
Epoch 24600/100000 Cost: 0.669193
Epoch 24700/100000 Cost: 0.669175
Epoch 24800/100000 Cost: 0.669157
Epoch 24900/100000 Cost: 0.669139
Epoch 25000/100000 Cost: 0.669122
Epoch 25100/100000 Cost: 0.669104
Epoch 25200/100000 Cost: 0.669087
Epoch 25300/100000 Cost: 0.669070
Epoch 25400/100000 Cost: 0.669053
Epoch 25500/100000 Cost: 0.669036
Epoch 25600/100000 Cost: 0.669019
Epoch 25700/100000 Cost: 0.669002
Epoch 25800/100000 Cost: 0.668986
Epoch 25900/100000 Cost: 0.668970
Epoch 26000/100000 Cost: 0.668953
Epoch 26100/100000 Cost: 0.668937
Epoch 26200/100000 Cost: 0.668921
Epoch 26300/100000 Cost: 0.668905
Epoch 26400/100000 Cost: 0.668889
Epoch 26500/100000 Cost: 0.668874
Epoch 26600/100000 Cost: 0.668858
Epoch 26700/100000 Cost: 0.668843
Epoch 26800/100000 Cost: 0.668827
Epoch 26900/100000 Cost: 0.668812
Epoch 27000/100000 Cost: 0.668797
Epoch 27100/100000 Cost: 0.668782
Epoch 27200/10

Epoch 48300/100000 Cost: 0.667009
Epoch 48400/100000 Cost: 0.667005
Epoch 48500/100000 Cost: 0.667000
Epoch 48600/100000 Cost: 0.666995
Epoch 48700/100000 Cost: 0.666991
Epoch 48800/100000 Cost: 0.666986
Epoch 48900/100000 Cost: 0.666982
Epoch 49000/100000 Cost: 0.666977
Epoch 49100/100000 Cost: 0.666973
Epoch 49200/100000 Cost: 0.666968
Epoch 49300/100000 Cost: 0.666963
Epoch 49400/100000 Cost: 0.666959
Epoch 49500/100000 Cost: 0.666955
Epoch 49600/100000 Cost: 0.666950
Epoch 49700/100000 Cost: 0.666946
Epoch 49800/100000 Cost: 0.666941
Epoch 49900/100000 Cost: 0.666937
Epoch 50000/100000 Cost: 0.666933
Epoch 50100/100000 Cost: 0.666928
Epoch 50200/100000 Cost: 0.666924
Epoch 50300/100000 Cost: 0.666919
Epoch 50400/100000 Cost: 0.666915
Epoch 50500/100000 Cost: 0.666911
Epoch 50600/100000 Cost: 0.666907
Epoch 50700/100000 Cost: 0.666902
Epoch 50800/100000 Cost: 0.666898
Epoch 50900/100000 Cost: 0.666894
Epoch 51000/100000 Cost: 0.666890
Epoch 51100/100000 Cost: 0.666885
Epoch 51200/10

Epoch 72300/100000 Cost: 0.666260
Epoch 72400/100000 Cost: 0.666258
Epoch 72500/100000 Cost: 0.666256
Epoch 72600/100000 Cost: 0.666254
Epoch 72700/100000 Cost: 0.666252
Epoch 72800/100000 Cost: 0.666250
Epoch 72900/100000 Cost: 0.666248
Epoch 73000/100000 Cost: 0.666246
Epoch 73100/100000 Cost: 0.666244
Epoch 73200/100000 Cost: 0.666242
Epoch 73300/100000 Cost: 0.666240
Epoch 73400/100000 Cost: 0.666238
Epoch 73500/100000 Cost: 0.666236
Epoch 73600/100000 Cost: 0.666234
Epoch 73700/100000 Cost: 0.666232
Epoch 73800/100000 Cost: 0.666230
Epoch 73900/100000 Cost: 0.666228
Epoch 74000/100000 Cost: 0.666226
Epoch 74100/100000 Cost: 0.666224
Epoch 74200/100000 Cost: 0.666222
Epoch 74300/100000 Cost: 0.666220
Epoch 74400/100000 Cost: 0.666218
Epoch 74500/100000 Cost: 0.666216
Epoch 74600/100000 Cost: 0.666214
Epoch 74700/100000 Cost: 0.666212
Epoch 74800/100000 Cost: 0.666210
Epoch 74900/100000 Cost: 0.666208
Epoch 75000/100000 Cost: 0.666206
Epoch 75100/100000 Cost: 0.666204
Epoch 75200/10

Epoch 96300/100000 Cost: 0.665886
Epoch 96400/100000 Cost: 0.665885
Epoch 96500/100000 Cost: 0.665883
Epoch 96600/100000 Cost: 0.665882
Epoch 96700/100000 Cost: 0.665881
Epoch 96800/100000 Cost: 0.665880
Epoch 96900/100000 Cost: 0.665879
Epoch 97000/100000 Cost: 0.665878
Epoch 97100/100000 Cost: 0.665877
Epoch 97200/100000 Cost: 0.665875
Epoch 97300/100000 Cost: 0.665874
Epoch 97400/100000 Cost: 0.665873
Epoch 97500/100000 Cost: 0.665872
Epoch 97600/100000 Cost: 0.665871
Epoch 97700/100000 Cost: 0.665870
Epoch 97800/100000 Cost: 0.665868
Epoch 97900/100000 Cost: 0.665867
Epoch 98000/100000 Cost: 0.665866
Epoch 98100/100000 Cost: 0.665865
Epoch 98200/100000 Cost: 0.665864
Epoch 98300/100000 Cost: 0.665863
Epoch 98400/100000 Cost: 0.665862
Epoch 98500/100000 Cost: 0.665861
Epoch 98600/100000 Cost: 0.665860
Epoch 98700/100000 Cost: 0.665858
Epoch 98800/100000 Cost: 0.665857
Epoch 98900/100000 Cost: 0.665856
Epoch 99000/100000 Cost: 0.665855
Epoch 99100/100000 Cost: 0.665854
Epoch 99200/10

# High-Level Implementation with nn.Module

In [52]:
class SoftmaxClassifierModel(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3 !!! 4개의 input을 받아 3개의 classes에 대한 확률값을 뱉어라
        
    def forward(self, x) : #어떻게 foward될꺼니?
        return self.linear(x)  # x=(m,4) -> linear(x) -> x=(m,3)

In [53]:
model = SoftmaxClassifierModel()

In [54]:
optimizer = optim.SGD(model.parameters(), lr=0.1)


nb_epochs = 1000

for epoch in range(nb_epochs + 1) :
    # H(x) 계산
    prediction = model(x_train)
    # x_train의 크기는 (8,4)
    # model을 통과한 x_train즉 
    # prediction 의 크기는 (8,3) 이 된다.
    
    
    # cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    # cost로 H(x1) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
     
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch,nb_epochs,cost.item()))
        

Epoch    0/1000 Cost: 2.019652
Epoch  100/1000 Cost: 0.672187
Epoch  200/1000 Cost: 0.579326
Epoch  300/1000 Cost: 0.522693
Epoch  400/1000 Cost: 0.477349
Epoch  500/1000 Cost: 0.437428
Epoch  600/1000 Cost: 0.400470
Epoch  700/1000 Cost: 0.364953
Epoch  800/1000 Cost: 0.329684
Epoch  900/1000 Cost: 0.293758
Epoch 1000/1000 Cost: 0.258090


훨씬더 적은 epoch를 줬는데도,, cost가 확연히 줄어든다.

In [51]:
prediction

tensor([[-4.6398, -0.7567,  4.2760],
        [-3.3405,  0.1046,  2.1106],
        [-7.6730,  2.7987,  4.0033],
        [-6.8134,  3.3480,  2.3364],
        [ 0.5110,  0.5405, -2.5438],
        [ 2.2043,  3.2920, -5.4020],
        [ 2.7883,  1.5669, -5.3705],
        [ 4.1860,  1.9692, -7.2001]], grad_fn=<AddmmBackward>)

Logistic Regression 과 굉장히 유사하다.  

하지만 
- Logistic Regression은 0과1의 두가지 discrete한 classes가 있는 classification 이라하면
- Softmax Classification의 경우엔 multinoial 확률분포가 됩니다. 

Binary Classification 의 경우 
- Binary Cross Entropy
- Sigmoid
를 사용하고,  

****

지금과 같이 여러개의 Classes가 있다면
- Cross Entropy
- softmax layer
를 사용한다.  